In [2]:
import numpy as np
import matplotlib.pyplot as plt # version > 3.0
import pandas as pd
from datetime import datetime
from google.colab import drive
import math
plt.style.use('seaborn')

In [3]:
path_to_sepsis3_single_stay_csv = "/content/Sepsis_Labs_Vitals_Demographic.csv"
sepsis_original = pd.read_csv(path_to_sepsis3_single_stay_csv)
print(sepsis_original.shape)
# sepsis_original.hist(column=["pao2fio2ratio_vent","respiration"])
# sepsis_original.hist(column=["pao2fio2ratio_novent","respiration"])
# sepsis_original.hist(column=["platelet_min","coagulation"])
# sepsis_original.hist(column=["bilirubin_max","liver"])
# sepsis_original.hist(column=["rate_dopamine","cardiovascular"])
# print(sepsis_original["rate_dopamine"].max())
# sepsis_original.hist(column=["rate_epinephrine","cardiovascular"])
# sepsis_original.hist(column=["rate_norepinephrine","cardiovascular"])
# sepsis_original.hist(column=["rate_dobutamine","cardiovascular"])
# sepsis_original.hist(column=["meanbp_min","cardiovascular"])
# sepsis_original.hist(column=["gcs_min","cns"])
# sepsis_original.hist(column=["creatinine_max","renal"])
# sepsis_original.hist(column=["uo_24hr","renal"])
# sepsis_original.hist(column="death_flag")
path_to_sepsis3_many_stays_csv = "/content/sepsis_labs_avg.csv"
path_to_vitals_stays_csv = "/content/vitals_avg.csv"
sepsis_many_stays_df = pd.read_csv(path_to_sepsis3_many_stays_csv)
vitals_df = pd.read_csv(path_to_vitals_stays_csv)
# print(sepsis_original.shape)
# print(sepsis_many_stays_df.shape)
# print(vitals_df.shape)
# sepsis_original.info()
# sepsis_original.describe()
# sepsis_many_stays_df.describe()
# sepsis_many_stays_df.info()

vitals_list = []

for column, _ in vitals_df.iteritems():
  vitals_list.append(column)

print(vitals_list)

sepsis_lab_list = []

for column, _ in sepsis_many_stays_df.iteritems():
  sepsis_lab_list.append(column)

print(sepsis_lab_list)

def find_first_non_null_sepsis_lab(subject_id, stay_id, column_name):
  modified_sepsis_lab_1 = sepsis_many_stays_df[sepsis_many_stays_df["subject_id"] == subject_id]
  modified_sepsis_lab_2 = modified_sepsis_lab_1[modified_sepsis_lab_1["stay_id"] == stay_id]
  for _, row in modified_sepsis_lab_2.iterrows():
    if type(row[column_name+"_avg"]) != str and not math.isnan(row[column_name+"_avg"]):
      return row[column_name+"_avg"]
    else:
      return None

def find_first_non_null_vital(subject_id, stay_id, column_name):
  modified_vitals_1 = vitals_df[vitals_df["subject_id"] == subject_id]
  modified_vitals_2 = modified_vitals_1[modified_vitals_1["stay_id"] == stay_id]
  for _, row in modified_vitals_2.iterrows():
    if type(row[column_name+"_avg"]) != str and not math.isnan(row[column_name+"_avg"]):
      return row[column_name+"_avg"]
    else:
      return None

sepsis_non_null_df = sepsis_original.copy(deep=True)

for column,_ in sepsis_original.iteritems():
  for i,row in sepsis_original.iterrows():
    if column+"_avg" in sepsis_lab_list and math.isnan(row[column]):
      non_null_value = find_first_non_null_sepsis_lab(row["subject_id"], row["stay_id"], column)
      if non_null_value is not None:
        print("Supposed to be non null: ",non_null_value)
      sepsis_non_null_df.at[i, column] = non_null_value
    elif column+"_avg" in vitals_list and math.isnan(row[column]):
      non_null_value = find_first_non_null_vital(row["subject_id"], row["stay_id"], column)
      if non_null_value is not None:
        print("Supposed to be non null: ",non_null_value)
      sepsis_non_null_df.at[i, column] = non_null_value
    else:
      continue

sepsis_non_null_df.info()
sepsis_non_null_df.to_csv("/content/Sepsis_Lab_and_Vital_with_Stay_Avg.csv")
  


In [ ]:
path_to_csv = "/content/Sepsis_Lab_and_Vital_with_Stay_Avg.csv"
updated_df = pd.read_csv(path_to_csv)
# print(updated_df.shape)
# updated_df.hist(column=["pao2fio2ratio_vent","respiration"])
# updated_df.hist(column=["pao2fio2ratio_novent","respiration"])
# updated_df.hist(column=["platelet_min","coagulation"])
# updated_df.hist(column=["bilirubin_max","liver"])
# updated_df.hist(column=["rate_dopamine","cardiovascular"])
# updated_df.hist(column=["rate_epinephrine","cardiovascular"])
# updated_df.hist(column=["rate_norepinephrine","cardiovascular"])
# updated_df.hist(column=["rate_dobutamine","cardiovascular"])
# updated_df.hist(column=["meanbp_min","cardiovascular"])
# updated_df.hist(column=["gcs_min","cns"])
# updated_df.hist(column=["creatinine_max","renal"])
# updated_df.hist(column=["uo_24hr","renal"])
# updated_df.hist(column="death_flag")
# updated_df.hist(column="sofa_score")
# updated_df.hist(column="sepsis3")
path_to_csv_with_both_sepsis_and_not_sepsis_icu_patients = "/content/Avg_sepsis_labs_both_Sepsis_and_not_Sepsis.csv"
new_df = pd.read_csv(path_to_csv_with_both_sepsis_and_not_sepsis_icu_patients)
# new_df[new_df["respiration_avg"] == 0].hist(column = ["pao2fio2ratio_novent_avg"])
# new_df[new_df["respiration_avg"] == 0].hist(column = ["pao2fio2ratio_vent_avg"])
# new_df[new_df["cardiovascular_avg"] == 0].hist(column = ["meanbp_min_avg"])
# new_df[new_df["renal_avg"] == 0].hist(column = ["uo_24hr_avg"])
# new_df[new_df["liver_avg"] == 0].hist(column = ["bilirubin_max_avg"])
# new_df[new_df["renal_avg"] == 0].hist(column = ["creatinine_max_avg"])
# new_df[new_df["coagulation_avg"] == 0].hist(column = ["platelet_min_avg"])
# vitals_df.describe(include='all').to_csv("vitals_summary.csv")
# vitals_df.hist(column=["heart_rate_avg"])
# vitals_df.hist(column=["sbp_avg"])
# vitals_df.hist(column=["mbp_avg"])
# vitals_df.hist(column=["dbp_avg"])
# vitals_df.hist(column=["sbp_ni_avg"])
# vitals_df.hist(column=["mbp_ni_avg"])
# vitals_df.hist(column=["dbp_ni_avg"])
# vitals_df.hist(column=["temperature_avg"])
# vitals_df.hist(column=["glucose_avg"])
# vitals_df.hist(column=["spo2_avg"])
# vitals_df.hist(column=["resp_rate_avg"])
pao2_novent_avg = new_df[new_df["respiration_avg"] == 0]["pao2fio2ratio_novent_avg"].median()
pao2_vent_avg = new_df[new_df["respiration_avg"] == 0]["pao2fio2ratio_vent_avg"].median()
meanbp_avg = new_df[new_df["cardiovascular_avg"] == 0]["meanbp_min_avg"].mean()
meanbp_std = new_df[new_df["cardiovascular_avg"] == 0]["meanbp_min_avg"].std()
uo_24hr_avg = new_df[new_df["renal_avg"] == 0]["uo_24hr_avg"].median()
bilr_avg = new_df[new_df["liver_avg"] == 0]["bilirubin_max_avg"].mean()
bilr_std = new_df[new_df["liver_avg"] == 0]["bilirubin_max_avg"].std()
creatinine_max_avg = new_df[new_df["renal_avg"] == 0]["creatinine_max_avg"].mean()
creatinine_max_std = new_df[new_df["renal_avg"] == 0]["creatinine_max_avg"].std()
platelet_min_avg = new_df[new_df["coagulation_avg"] == 0]["platelet_min_avg"].median()
heart_rate_avg = vitals_df["heart_rate_avg"].mean()
heart_rate_std = vitals_df["heart_rate_avg"].std()
sbp_avg = vitals_df["sbp_avg"].mean()
sbp_std = vitals_df["sbp_avg"].std()
mbp_avg = vitals_df["mbp_avg"].mean()
mbp_std = vitals_df["mbp_avg"].std()
dbp_avg = vitals_df["dbp_avg"].mean()
dbp_std = vitals_df["dbp_avg"].std()
sbp_ni_avg = vitals_df["sbp_ni_avg"].mean()
sbp_ni_std = vitals_df["sbp_ni_avg"].std()
mbp_ni_avg = vitals_df["mbp_ni_avg"].mean()
mbp_ni_std = vitals_df["mbp_ni_avg"].std()
dbp_ni_avg = vitals_df["dbp_ni_avg"].mean()
dbp_ni_std = vitals_df["dbp_ni_avg"].std()
temperature_avg = vitals_df["temperature_avg"].mean()
temperature_std = vitals_df["temperature_avg"].std()
glucose_avg = vitals_df["glucose_avg"].median()
spo2_avg = vitals_df["spo2_avg"].mean()
spo2_std = vitals_df["spo2_avg"].std()
resp_rate_avg = vitals_df["resp_rate_avg"].mean()
resp_rate_std = vitals_df["resp_rate_avg"].std()
updated_df_copy = pd.read_csv("/content/try-avg.csv")
# print(updated_df_copy.shape)
# updated_df.info()
for row_idx, row in updated_df.iterrows():
  print(row_idx)
  row_updated = row.fillna(value={"pao2fio2ratio_novent": float(np.random.uniform(pao2_novent_avg -50 , pao2_novent_avg + 50)),
                                                  "pao2fio2ratio_vent": float(np.random.uniform(pao2_vent_avg - 50 , pao2_vent_avg + 50)),
                                                  "rate_dobutamine":0 ,
                                                  "rate_epinephrine":0,
                                                  "rate_norepinephrine":0, 
                                                  "rate_dopamine":0,
                                                  "meanbp_min":float(np.random.normal(meanbp_avg , meanbp_std)),
                                                  "gcs_min": 15.0, 
                                                  "uo_24hr": float(np.random.uniform(uo_24hr_avg - 100, uo_24hr_avg + 100)),
                                                  "bilirubin_max": float(np.random.normal(bilr_avg, bilr_std)),
                                                  "creatinine_max":float(np.random.normal(creatinine_max_avg, creatinine_max_std)),
                                                  "platelet_min":float(np.random.uniform(platelet_min_avg - 50 , platelet_min_avg + 50)),
                                                  "heart_rate":float(np.random.normal(heart_rate_avg, heart_rate_std)), 
                                                  "sbp":float(np.random.normal(sbp_avg, sbp_std)), 
                                                  "mbp":float(np.random.normal(mbp_avg, mbp_std)), 
                                                  "dbp":float(np.random.normal(dbp_avg, dbp_std)), 
                                                  "sbp_ni":float(np.random.normal(sbp_ni_avg, sbp_ni_std)), 
                                                  "mbp_ni":float(np.random.normal(mbp_ni_avg, mbp_ni_std)),
                                                  "dbp_ni":float(np.random.normal(dbp_ni_avg, dbp_ni_std)), 
                                                  "temperature":float(np.random.normal(temperature_avg, temperature_std)),
                                                  "glucose":float(np.random.uniform(glucose_avg - 10, glucose_avg + 10)),
                                                  "spo2":float(np.random.normal(spo2_avg, spo2_std)),
                                                  "resp_rate":float(np.random.normal(resp_rate_avg, resp_rate_std))})
  updated_df_copy = updated_df_copy.append(row_updated)
  # print(updated_df_copy.head())
  # print(updated_df_copy.shape)

# updated_df_copy.info()
# print(updated_df_copy.shape)
updated_df_copy.to_csv("/content/No_NaN_Values_Newest_14May_1.csv")


In [ ]:
updated_df_1 = pd.read_csv("/content/No_NaN_Values_Newest_14May_1.csv")
updated_df_1.info()